## Check if Instance is Present

In [1]:
import boto3

ec2 = boto3.client('ec2')

response = ec2.describe_instances()

instance_name = "prahalad_ec2"

instance_id="" 

for resp in response['Reservations']:
    resp = resp['Instances'][0]
    tags = resp.get('Tags', [])
    
    for tag in tags:
        if tag.get("Key", "")=="Name" and tag.get("Value", "")==instance_name:
            instance_id = resp['InstanceId']

if instance_id=="":
    print(f"No instance found with name {instance_name}")

instance_id

No instance found with name prahalad_ec2


''

## Create an Amazon EC2 instance

In [3]:
import boto3

ec2 = boto3.client('ec2')

if instance_id == "":
    response = ec2.run_instances(
        ImageId = 'ami-0772e0bfc7cc9c5de',
        MinCount=1,
        MaxCount=1,
        InstanceType='t2.large',
        KeyName='Prahalad_ReddyKEY',
        BlockDeviceMappings=[
            {
                "DeviceName": "/dev/xvda",
                'Ebs':{
                    'DeleteOnTermination': True,
                    'VolumeSize': 120
                }
            }
        ]

    )

    instance_id = response['Instances'][0]['InstanceId']

    ec2.create_tags(Resources=[instance_id], Tags=[
        {
            'Key':'Name',
            'Value':instance_name
        }
    ])

else:
    print("Instance is already present")

## Create Security Group and add rules to it
- Security groups control inbound and outbound traffic of the EC2 instance network interface.
- every EC2 instance must have at least one Security Group associated with it. If no Security Group has been specified during the EC2 instance launch, the default Security Group of the default VPC is associated with the instance.

In [4]:
group_name = 'prahald'

response = ec2.describe_security_groups()

security_group_id = [x['GroupId'] for x in response['SecurityGroups'] if x['GroupName']==group_name]

if security_group_id == []:
    response = ec2.create_security_group(
        GroupName = group_name,
        Description = "Security group for testing"
    )
    security_group_id = response['GroupId']
else:
    security_group_id = security_group_id[0]

security_group_id

'sg-0fb8d4cfecc64d7e5'

In [5]:
from botocore.exceptions import ClientError

def update_security_group(group_id, protocol, port, cidr):
    try:
        response = ec2.authorize_security_group_ingress(
            GroupId = group_id,
            IpPermissions=[
                {
                    'IpProtocol': protocol,
                    'FromPort': port,
                    'ToPort': port,
                    'IpRanges': [{'CidrIp': cidr}]
                }
            ]
        )
    except ClientError as e:
        if e.response['Error']['Code']=='InvalidPermission.Duplicate':
            print('This rule is already there')
        else:
            print("an error as occured!")
            print(e)

update_security_group(security_group_id, 'tcp', 22, '0.0.0.0/0')
update_security_group(security_group_id, 'tcp', 80, '0.0.0.0/0')
update_security_group(security_group_id, 'tcp', 8501, '0.0.0.0/0')
update_security_group(security_group_id, 'tcp', 8502, '0.0.0.0/0')

In [6]:
ec2.modify_instance_attribute(InstanceId=instance_id, Groups=[security_group_id])

{'ResponseMetadata': {'RequestId': 'be8a5b4a-e5e9-487d-bd4b-a243305a7e95',
  'HTTPStatusCode': 200,
  'HTTPHeaders': {'x-amzn-requestid': 'be8a5b4a-e5e9-487d-bd4b-a243305a7e95',
   'cache-control': 'no-cache, no-store',
   'strict-transport-security': 'max-age=31536000; includeSubDomains',
   'content-type': 'text/xml;charset=UTF-8',
   'content-length': '235',
   'date': 'Tue, 31 Dec 2024 11:32:21 GMT',
   'server': 'AmazonEC2'},
  'RetryAttempts': 0}}

## Attach S3 Policy to EC2 Instance

In [6]:
# describe IAM role
iam = boto3.client('iam')

role_name = "ec2-s3-full-access"

response = iam.get_role(RoleName=role_name)

role_arn = response['Role']['Arn']

role_arn

# Ensure there is an instance profile with the same name as the role
instance_profile_name = role_name
try:
    iam.get_instance_profile(InstanceProfileName=instance_profile_name)
except iam.exceptions.NoSuchEntityException:
    # Create an instance profile if it doesn't exist
    iam.create_instance_profile(InstanceProfileName=instance_profile_name)
    # Add role to the instance profile
    iam.add_role_to_instance_profile(
        InstanceProfileName=instance_profile_name,
        RoleName=role_name
    )

# Attach the instance profile to the EC2 instance
ec2.associate_iam_instance_profile(
    IamInstanceProfile={
        'Name': instance_profile_name
    },
    InstanceId=instance_id
)

{'IamInstanceProfileAssociation': {'AssociationId': 'iip-assoc-005620e6c249a116f',
  'InstanceId': 'i-00ce7804abca18c47',
  'IamInstanceProfile': {'Arn': 'arn:aws:iam::539320159019:instance-profile/ec2-s3-full-access',
   'Id': 'AIPAX3EP4RMVWCBMURPGY'},
  'State': 'associating'},
 'ResponseMetadata': {'RequestId': '3811198f-03fe-4b9f-9e39-838301d6de88',
  'HTTPStatusCode': 200,
  'HTTPHeaders': {'x-amzn-requestid': '3811198f-03fe-4b9f-9e39-838301d6de88',
   'cache-control': 'no-cache, no-store',
   'strict-transport-security': 'max-age=31536000; includeSubDomains',
   'vary': 'accept-encoding',
   'content-type': 'text/xml;charset=UTF-8',
   'transfer-encoding': 'chunked',
   'date': 'Wed, 07 Aug 2024 08:36:57 GMT',
   'server': 'AmazonEC2'},
  'RetryAttempts': 0}}

### Terminate

In [ ]:
import time

def wait_for_status(instance_id, target_status):
    while True:
        response = ec2.describe_instances(InstanceIds=instance_id)

        status = response['Reservations'][0]['Instances'][0]['State']['Name']

        if status == target_status:
            print("Instance is in {} state".format(target_status))
            break
        
        time.sleep(10)

def terminate_instances(instance_id):
    print("EC2 Instance Termination")
    ec2.terminate_instances(InstanceIds=instance_id)

    wait_for_status(instance_id, 'terminated')

# terminate_instances([instance_id])